In [1]:
import pandas as pd

In [2]:
X = pd.read_json("../data/train_data.json", orient = "records")

In [25]:
X.head()

,aspiration,bodyStyle,bore,cityMPG,compressionRatio,curbWeight,driveWheels,engineLocation,engineSize,engineType,...,length,make,numCylinders,numDoors,peakRPM,price,risk,stroke,wheelBase,width
0,std,convertible,3.47,21,9.0,2548,rwd,front,130,dohc,...,168.8,alfa-romero,four,two,5000.0,13495.0,3,2.68,88.6,64.1
1,std,convertible,3.47,21,9.0,2548,rwd,front,130,dohc,...,168.8,alfa-romero,four,two,5000.0,16500.0,3,2.68,88.6,64.1
2,std,hatchback,2.68,19,9.0,2823,rwd,front,152,ohcv,...,171.2,alfa-romero,six,two,5000.0,16500.0,1,3.47,94.5,65.5
3,std,sedan,3.19,18,8.0,2824,4wd,front,136,ohc,...,176.6,audi,five,four,5500.0,17450.0,2,3.40,99.4,66.4
4,std,sedan,3.19,19,8.5,2507,fwd,front,136,ohc,...,177.3,audi,five,two,5500.0,15250.0,2,3.40,99.8,66.3


In [26]:
X.select_dtypes(include=['object']).head()

,aspiration,bodyStyle,driveWheels,engineLocation,engineType,fuelSystem,fuelType,make,numCylinders,numDoors
0,std,convertible,rwd,front,dohc,mpfi,gas,alfa-romero,four,two
1,std,convertible,rwd,front,dohc,mpfi,gas,alfa-romero,four,two
2,std,hatchback,rwd,front,ohcv,mpfi,gas,alfa-romero,six,two
3,std,sedan,4wd,front,ohc,mpfi,gas,audi,five,four
4,std,sedan,fwd,front,ohc,mpfi,gas,audi,five,two


In [27]:
X.select_dtypes(exclude=['object']).head()

,bore,cityMPG,compressionRatio,curbWeight,engineSize,height,highwayMPG,horsepower,length,peakRPM,price,risk,stroke,wheelBase,width
0,3.47,21,9.0,2548,130,48.8,27,111.0,168.8,5000.0,13495.0,3,2.68,88.6,64.1
1,3.47,21,9.0,2548,130,48.8,27,111.0,168.8,5000.0,16500.0,3,2.68,88.6,64.1
2,2.68,19,9.0,2823,152,52.4,26,154.0,171.2,5000.0,16500.0,1,3.47,94.5,65.5
3,3.19,18,8.0,2824,136,54.3,22,115.0,176.6,5500.0,17450.0,2,3.40,99.4,66.4
4,3.19,19,8.5,2507,136,53.1,25,110.0,177.3,5500.0,15250.0,2,3.40,99.8,66.3


In [22]:
X.dtypes

aspiration           object
bodyStyle            object
bore                float64
cityMPG               int64
compressionRatio    float64
curbWeight            int64
driveWheels          object
engineLocation       object
engineSize            int64
engineType           object
fuelSystem           object
fuelType             object
height              float64
highwayMPG            int64
horsepower          float64
length              float64
make                 object
numCylinders         object
numDoors             object
peakRPM             float64
price               float64
risk                  int64
stroke              float64
wheelBase           float64
width               float64
dtype: object

In [15]:
X.select_dtypes(include=['object']).columns.sort

<bound method Index.sort of Index([u'aspiration', u'bodyStyle', u'driveWheels', u'engineLocation',
       u'engineType', u'fuelSystem', u'fuelType', u'make', u'numCylinders',
       u'numDoors'],
      dtype='object')>

In [14]:
X.select_dtypes(exclude=['object']).columns.sort

<bound method Index.sort of Index([u'bore', u'cityMPG', u'compressionRatio', u'curbWeight', u'engineSize',
       u'height', u'highwayMPG', u'horsepower', u'length', u'peakRPM',
       u'price', u'risk', u'stroke', u'wheelBase', u'width'],
      dtype='object')>

In [13]:
X.columns.sort

<bound method Index.sort of Index([u'aspiration', u'bodyStyle', u'bore', u'cityMPG', u'compressionRatio',
       u'curbWeight', u'driveWheels', u'engineLocation', u'engineSize',
       u'engineType', u'fuelSystem', u'fuelType', u'height', u'highwayMPG',
       u'horsepower', u'length', u'make', u'numCylinders', u'numDoors',
       u'peakRPM', u'price', u'risk', u'stroke', u'wheelBase', u'width'],
      dtype='object')>

## Testing new helper class

In [1]:
import json
import numpy as np
import pandas as pd
from score_auto_gbm.PipelinePersister import PipelinePersister
from score_auto_gbm.ModelEvaluator import ModelEvaluator

def score(record, model):
    datum = json.loads(record)
    score = list(model.predict(pd.DataFrame([datum])))[0]
    return json.dumps(score)

In [2]:
train_data = pd.read_json("../data/train_data.json", orient = "records")

X = train_data[['engineType', 'fuelType', 'driveWheels', 'curbWeight', 'numDoors', 'wheelBase']]
y = np.array(train_data["risk"])

model, preprocessor = PipelinePersister().fit(X, y, useGridSearch=True)

evaluator = ModelEvaluator()

###Use result = pd.concat([df1,df2,df3]). df1 = pd.DataFrame([json.load(line)])
##with open(path, 'rb') as f -> for line in f
eval_data = pd.read_json("../data/gbm_input_data_multiline.json", orient = "records")
X_eval = eval_data.ix[:, 1:]
y_eval = np.array(eval_data['risk'])

rmse, standard_rmse, total = evaluator.get_rmse_summary(evaluator.get_predict_vs_actuals(model, X_eval, y_eval, preprocessor))
print('With Standardizeation:\nRMSE: %f.\nSTD RMSE: %f.\nTotal: %d.' % (rmse,standard_rmse, total))

rmse, standard_rmse, total = evaluator.get_rmse_summary(evaluator.get_predict_vs_actuals(model, X_eval, y_eval))
print('Without Standardizeation:\nRMSE: %f.\nSTD RMSE: %f.\nTotal: %d.' % (rmse,standard_rmse, total))

ValueError: Trailing data

In [4]:
PipelinePersister().save(test, '../models/gbmFit-best6features.pkl')